In [19]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [20]:
L, W = 1.0, 0.5  # Length and width of the domain (in cm)
lambda_ = 5.0e9   # Elastic constant (Pa)
mu = 5.0e9         # Shear modulus (Pa)
h = 1.0          # Thickness (cm)

In [21]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        return self.net(inputs)

In [22]:
def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True, allow_unused=True)[0]
    u_x_y = torch.autograd.grad(u_x, y, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

    E_xx = u_x_x
    E_yy = u_y_y
    E_xy = 0.5 * (u_x_y + u_y_x)
    return E_xx, E_yy, E_xy

In [23]:
def stress_tensor(E_xx, E_yy, E_xy):
    scale_factor = 1e9  # Scaling factor for material constants
    trace_E = E_xx + E_yy
    sigma_xx = h * ((lambda_ / scale_factor) * trace_E + 2 * (mu / scale_factor) * E_xx)
    sigma_yy = h * ((lambda_ / scale_factor) * trace_E + 2 * (mu / scale_factor) * E_yy)
    sigma_xy = h * (2 * (mu / scale_factor) * E_xy)
    return sigma_xx, sigma_yy, sigma_xy

In [24]:
def physics_loss(model, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)
    u = model(x, y)
    u_x, u_y = u[:, 0:1], u[:, 1:2]

    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x, y)
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    sigma_xx_x = torch.autograd.grad(sigma_xx, x, grad_outputs=torch.ones_like(sigma_xx), retain_graph=True, create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y, grad_outputs=torch.ones_like(sigma_yy), retain_graph=True, create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]

    residual_x = sigma_xx_x + sigma_xy_y
    residual_y = sigma_yy_y + sigma_xy_x

    loss_equilibrium = torch.mean(residual_x**2 + residual_y**2)
    return loss_equilibrium

In [25]:
def boundary_condition_loss(model, L, W):
    # Boundary A: u_x = 0, u_y = 0 at x = -L/2
    y_A = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    u_A = model(-L / 2 * torch.ones_like(y_A, requires_grad=True), y_A)
    loss_A = torch.mean(u_A**2)

    # Boundary D: u_x = 0.025 * L, u_y = 0 at x = L/2
    y_D = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    u_D = model(L / 2 * torch.ones_like(y_D, requires_grad=True), y_D)
    loss_D = torch.mean((u_D[:, 1:2]**2) + (u_D[:, 0:1] - 0.025 * L)**2)

    # Boundary B: traction-free (σ_xx = σ_xy = 0) at y = W/2
    x_B = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_B = W / 2 * torch.ones_like(x_B, requires_grad=True)
    u_B = model(x_B, y_B)
    u_B_x, u_B_y = u_B[:, 0:1], u_B[:, 1:2]
    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_B_x, u_B_y, x_B, y_B)
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    loss_B = torch.mean(sigma_yy_B**2 +sigma_xy_B**2)

    # Boundary C: traction-free (σ_xx = σ_xy = 0) at y = -W/2
    x_C = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_C = -W / 2 * torch.ones_like(x_C, requires_grad=True)
    u_C = model(x_C, y_C)
    u_C_x, u_C_y = u_C[:, 0:1], u_C[:, 1:2]
    E_xx_C, E_yy_C, E_xy_C = strain_tensor(u_C_x, u_C_y, x_C, y_C)
    sigma_xx_C, sigma_yy_C, sigma_xy_C = stress_tensor(E_xx_C, E_yy_C, E_xy_C)
    loss_C = torch.mean(sigma_yy_C**2 + sigma_xy_C**2)

    return loss_A + loss_D + loss_B / 10**18 + loss_C / 10**18

In [26]:
def train_pinn(model, optimizer, n_epochs, n_points, L, W):
    loss_history = []

    for epoch in range(n_epochs):
        # Sample points inside the domain
        x = torch.rand((n_points, 1)) * L - L / 2 # make kinda of mesh of xy xy and 
        y = torch.rand((n_points, 1)) * W - W / 2

        loss_pde = physics_loss(model, x, y)
        loss_bc = boundary_condition_loss(model, L, W)
        loss = loss_pde + loss_bc 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    return loss_history

In [27]:
# Initialize model and optimizer
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
n_epochs = 10000
n_points = 1000
loss_history = train_pinn(model, optimizer, n_epochs, n_points, L, W)


Epoch 0, Loss: 0.178013
Epoch 500, Loss: 0.000013
Epoch 1000, Loss: 0.000002
Epoch 1500, Loss: 0.000002
Epoch 2000, Loss: 0.000024
Epoch 2500, Loss: 0.000247
Epoch 3000, Loss: 0.000000
Epoch 3500, Loss: 0.000001
Epoch 4000, Loss: 0.000000
Epoch 4500, Loss: 0.000001
Epoch 5000, Loss: 0.000000
Epoch 5500, Loss: 0.000000
Epoch 6000, Loss: 0.000000
Epoch 6500, Loss: 0.000055
Epoch 7000, Loss: 0.000007
Epoch 7500, Loss: 0.000000
Epoch 8000, Loss: 0.000000
Epoch 8500, Loss: 0.000011
Epoch 9000, Loss: 0.000000
Epoch 9500, Loss: 0.000000


In [42]:
import pandas as pd
from sklearn.metrics import mean_squared_error 

In [43]:
file_path = '/Users/murat/Downloads/data.csv'  # Update this with the correct path
comparison_data = pd.read_csv(file_path)

In [44]:
x_values = torch.tensor(comparison_data['X'].values, dtype=torch.float32).reshape(-1, 1)
y_values = torch.tensor(comparison_data['Y'].values, dtype=torch.float32).reshape(-1, 1)
u_x_actual = comparison_data['u_x_actual'].values
u_y_actual = comparison_data['u_y_actual'].values

In [45]:
model.eval()
with torch.no_grad():
    u_pred = model(x_values, y_values) 
    u_x_pred = u_pred[:, 0].reshape(-1).numpy()
    u_y_pred = u_pred[:, 1].reshape(-1).numpy()

In [46]:
comparison_data['u_x_pred'] = u_x_pred
comparison_data['u_y_pred'] = u_y_pred

In [47]:
comparison_data['error_u_x'] = abs(comparison_data['u_x_actual'] - comparison_data['u_x_pred'])
comparison_data['error_u_y'] = abs(comparison_data['u_y_actual'] - comparison_data['u_y_pred'])
comparison_data['percent_error_u_x'] = mean_squared_error(u_x_actual,u_x_pred) 
comparison_data['percent_error_u_y'] = mean_squared_error(u_y_actual,u_y_pred) 


In [48]:
print(comparison_data)

         X     Y    u_x_actual    u_y_actual  u_x_pred  u_y_pred  error_u_x  \
0    -0.50 -0.25  1.483633e-19  1.263433e-19 -0.000664  0.000586   0.000664   
1    -0.48 -0.25  9.101041e-04  9.363998e-04 -0.000170  0.000590   0.001080   
2    -0.50 -0.23 -1.530000e-20 -4.190000e-20 -0.000600  0.000528   0.000600   
3    -0.48 -0.23  5.332043e-04  6.506182e-04 -0.000107  0.000531   0.000640   
4    -0.50 -0.21  2.321675e-21 -6.630000e-21 -0.000536  0.000470   0.000536   
...    ...   ...           ...           ...       ...       ...        ...   
1321  0.48  0.23  2.446680e-02 -6.510000e-04  0.025158 -0.000773   0.000691   
1322  0.50  0.21  2.500000e-02  3.680167e-18  0.025598 -0.000668   0.000598   
1323  0.48  0.25  2.408990e-02 -9.360000e-04  0.025215 -0.000850   0.001125   
1324  0.50  0.23  2.500000e-02  1.768693e-18  0.025656 -0.000745   0.000656   
1325  0.50  0.25  2.500000e-02  2.080774e-17  0.025713 -0.000823   0.000713   

      error_u_y  percent_error_u_x  percent_error_u

In [35]:
comparison_data.to_csv('/Users/murat/Downloads/comparison_results.csv', index=False)